In [1]:
from ease4lmp import BondedAtoms, LammpsWriter
from ase import Atoms
from ase.build import bulk, molecule

In [2]:
import itertools
import numpy as np

## Methanol

In [3]:
atoms = BondedAtoms(molecule("CH3OH"))

In [4]:
atoms.get_atomic_numbers()

array([6, 8, 1, 1, 1, 1])

In [5]:
print("O-H", atoms.get_distance(1, 3))
print("C-H", atoms.get_distance(0, 2))
print("C-H", atoms.get_distance(0, 4))
print("C-H", atoms.get_distance(0, 5))

O-H 0.9700009076665858
C-H 1.0895403725020931
C-H 1.0967341192508784
C-H 1.0967341192508784


In [6]:
atoms.set_types([1, 2, 3, 4, 3, 3])

Create bonds connecting two atoms separated by less than 1.5 angstroms.

In [7]:
positions = atoms.get_positions()
bond_list = [
  (i, j)
  for i, j in itertools.combinations(range(len(atoms)), 2)
  if np.linalg.norm(positions[i] - positions[j]) < 1.5
]
bond_list

[(0, 1), (0, 2), (0, 4), (0, 5), (1, 3)]

In [8]:
for pair in bond_list:
  atoms.add_bond(*pair)

In [9]:
atoms.get_types()

array([1, 2, 3, 4, 3, 3])

In [10]:
atoms.get_bonds()

array([[[ 1,  0,  0,  0],
        [ 2,  0,  0,  0],
        [ 4,  0,  0,  0],
        [ 5,  0,  0,  0]],

       [[-1,  0,  0,  0],
        [ 2,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[-2,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[-2,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[-4,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[-5,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]]])

In [11]:
atoms.get_positions()

array([[-0.047131,  0.664389,  0.      ],
       [-0.047131, -0.758551,  0.      ],
       [-1.092995,  0.969785,  0.      ],
       [ 0.878534, -1.048458,  0.      ],
       [ 0.437145,  1.080376,  0.891772],
       [ 0.437145,  1.080376, -0.891772]])

Unit cell is required to write Lammps' file.

In [12]:
atoms.set_cell([[5., 0., 0.], [0., 5., 0.], [0., 0., 5.]])
atoms.center()

In [13]:
atoms.get_positions()

array([[2.5600995, 3.14843  , 2.5      ],
       [2.5600995, 1.72549  , 2.5      ],
       [1.5142355, 3.453826 , 2.5      ],
       [3.4857645, 1.435583 , 2.5      ],
       [3.0443755, 3.564417 , 3.391772 ],
       [3.0443755, 3.564417 , 1.608228 ]])

Set random velocities to all the atoms.

In [14]:
atoms.set_velocities(np.random.uniform(-1, 1, (len(atoms), 3)))

Create a LammpsWriter instance.

In [15]:
writer = LammpsWriter(atoms, atom_style="molecular")

LammpsAtoms: 'id' have been set
LammpsAtoms: 'type' have been set
LammpsAtoms: 'x' have been set
LammpsAtoms: 'y' have been set
LammpsAtoms: 'z' have been set
LammpsAtoms: 'vx' have been set
LammpsAtoms: 'vy' have been set
LammpsAtoms: 'vz' have been set
LammpsAtoms: 'mass' have been set
LammpsBonds: 'id' have been set
LammpsBonds: 'atom1' have been set
LammpsBonds: 'atom2' have been set
LammpsAngles: 'id' have been set
LammpsAngles: 'atom1' have been set
LammpsAngles: 'atom2' have been set
LammpsAngles: 'atom3' have been set
LammpsDihedrals: 'id' have been set
LammpsDihedrals: 'atom1' have been set
LammpsDihedrals: 'atom2' have been set
LammpsDihedrals: 'atom3' have been set
LammpsDihedrals: 'atom4' have been set
LammpsImpropers: 'id' have been set
LammpsImpropers: 'atom1' have been set
LammpsImpropers: 'atom2' have been set
LammpsImpropers: 'atom3' have been set
LammpsImpropers: 'atom4' have been set


Check missing data.

In [16]:
writer.get_required_datanames()

{'mol'}

In [17]:
writer.get_required_datanames_for_molecule()

set()

All the atoms belong to the same molecule.
Consider that the molecule's id is 1.

In [18]:
writer.set_atom_data(mol=[1]*len(atoms))

LammpsAtoms: 'mol' have been set


Set types of bond, angle, dihedral and improper.

In [19]:
bond_types = {
  seq: i+1
  for i, seq in enumerate(writer.get_bond_patterns())
}
bond_types

{(1, 2): 1, (1, 3): 2, (2, 4): 3}

In [20]:
writer.set_bond_types(bond_types)

LammpsBonds: 'type' have been set


In [21]:
angle_types = {
  seq: i+1
  for i, seq in enumerate(writer.get_angle_patterns())
}
angle_types

{(3, 1, 3): 1, (2, 1, 3): 2, (1, 2, 4): 3}

In [22]:
writer.set_angle_types(angle_types)

LammpsAngles: 'type' have been set


In [23]:
dihedral_types = {
  seq: i+1
  for i, seq in enumerate(writer.get_dihedral_patterns())
}
dihedral_types

{(4, 2, 1, 3): 1}

In [24]:
writer.set_dihedral_types(dihedral_types)

LammpsDihedrals: 'type' have been set


In [25]:
improper_types = {
  seq: i+1
  for i, seq in enumerate(writer.get_improper_patterns())
}
improper_types

{(1, 2, 3, 3): 1, (1, 3, 3, 3): 2}

In [26]:
writer.set_improper_types(improper_types)

LammpsImpropers: 'type' have been set


The following methods print information about keywords of Lammps' `read_data` command.
In the below case, the `read_data` command will be something like:

```python
from lammps import PyLammps

L = PyLammps()
L.read_data(
    "path/to/datafile",
    "extra/bond/per/atom", 4,
    "extra/angle/per/atom", 6,
    "extra/dihedral/per/atom", 2,
    "extra/improper/per/atom", 3)
```

In [27]:
writer.print_max_bonds_per_atom()
writer.print_max_angles_per_atom()
writer.print_max_dihedrals_per_atom()
writer.print_max_impropers_per_atom()

You might need to set 'extra/bond/per/atom' to: 4
You might need to set 'extra/angle/per/atom' to: 6
You might need to set 'extra/dihedral/per/atom' to: 2
You might need to set 'extra/improper/per/atom' to: 3


Write Lammps' data file with *Masses* section.

In [28]:
writer.write_lammps_data("./lammps_files/data.methanol", mass=True)

'Masses' section was written
'Atoms' section was written
'Bonds' section was written
'Angles' section was written
'Dihedrals' section was written
'Impropers' section was written


* [link to the file](./lammps_files/data.methanol)

Write Lammps' molecule file with *Masses* section.

In [29]:
writer.write_lammps_molecule("./lammps_files/molecule.methanol", mass=True)

'Coords' section was written
'Types' section was written
'Masses' section was written
'Bonds' section was written
'Angles' section was written
'Dihedrals' section was written
'Impropers' section was written
'Special Bond Counts' and 'Special Bonds' section was written


* [link to the file](./lammps_files/molecule.methanol)

Write Lammps' data file with *Masses* and *velocities* section.
In addition, `centering=True` ensures that center of the simulation box is at $(0, 0, 0)$.

In [30]:
writer.write_lammps_data("./lammps_files/data.methanol2", centering=True, velocity=True, mass=True)

'Masses' section was written
'Atoms' section was written
'Velocities' section was written
'Bonds' section was written
'Angles' section was written
'Dihedrals' section was written
'Impropers' section was written


* [link to the file](./lammps_files/data.methanol2)

## Acetic acid

In [31]:
atoms = BondedAtoms(molecule("CH3COOH"))

In [32]:
atoms.get_atomic_numbers()

array([6, 8, 8, 1, 6, 1, 1, 1])

In [33]:
print("O-H", atoms.get_distance(2, 3))
print("C-H", atoms.get_distance(4, 5))
print("C-H", atoms.get_distance(4, 6))
print("C-H", atoms.get_distance(4, 7))

O-H 0.9792030242666737
C-H 1.0882694768934758
C-H 1.0920259389130829
C-H 1.0920259389130829


In [34]:
atoms.set_types([1, 2, 3, 4, 5, 6, 6, 6])

In [35]:
positions = atoms.get_positions()
bond_list = [
  (i, j)
  for i, j in itertools.combinations(range(len(atoms)), 2)
  if np.linalg.norm(positions[i] - positions[j]) < 1.5
]
bond_list

[(0, 1), (0, 2), (0, 4), (2, 3), (4, 5), (4, 6), (4, 7)]

In [36]:
for pair in bond_list:
  atoms.add_bond(*pair)

In [37]:
atoms.get_types()

array([1, 2, 3, 4, 5, 6, 6, 6])

In [38]:
atoms.get_bonds()

array([[[ 1,  0,  0,  0],
        [ 2,  0,  0,  0],
        [ 4,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[-1,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[-2,  0,  0,  0],
        [ 1,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[-1,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[-4,  0,  0,  0],
        [ 1,  0,  0,  0],
        [ 2,  0,  0,  0],
        [ 3,  0,  0,  0]],

       [[-1,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[-2,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[-3,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]]])

In [39]:
atoms.get_positions()

array([[ 0.      ,  0.15456 ,  0.      ],
       [ 0.166384,  1.360084,  0.      ],
       [-1.236449, -0.415036,  0.      ],
       [-1.867646,  0.333582,  0.      ],
       [ 1.073776, -0.892748,  0.      ],
       [ 2.048189, -0.408135,  0.      ],
       [ 0.968661, -1.528353,  0.881747],
       [ 0.968661, -1.528353, -0.881747]])

In [40]:
atoms.set_cell([[5., 0., 0.], [0., 5., 0.], [0., 0., 5.]])
atoms.center()

In [41]:
atoms.get_positions()

array([[2.4097285, 2.7386945, 2.5      ],
       [2.5761125, 3.9442185, 2.5      ],
       [1.1732795, 2.1690985, 2.5      ],
       [0.5420825, 2.9177165, 2.5      ],
       [3.4835045, 1.6913865, 2.5      ],
       [4.4579175, 2.1759995, 2.5      ],
       [3.3783895, 1.0557815, 3.381747 ],
       [3.3783895, 1.0557815, 1.618253 ]])

In [42]:
atoms.set_velocities(np.random.uniform(-1, 1, (len(atoms), 3)))

In [43]:
writer = LammpsWriter(atoms, atom_style="molecular")

LammpsAtoms: 'id' have been set
LammpsAtoms: 'type' have been set
LammpsAtoms: 'x' have been set
LammpsAtoms: 'y' have been set
LammpsAtoms: 'z' have been set
LammpsAtoms: 'vx' have been set
LammpsAtoms: 'vy' have been set
LammpsAtoms: 'vz' have been set
LammpsAtoms: 'mass' have been set
LammpsBonds: 'id' have been set
LammpsBonds: 'atom1' have been set
LammpsBonds: 'atom2' have been set
LammpsAngles: 'id' have been set
LammpsAngles: 'atom1' have been set
LammpsAngles: 'atom2' have been set
LammpsAngles: 'atom3' have been set
LammpsDihedrals: 'id' have been set
LammpsDihedrals: 'atom1' have been set
LammpsDihedrals: 'atom2' have been set
LammpsDihedrals: 'atom3' have been set
LammpsDihedrals: 'atom4' have been set
LammpsImpropers: 'id' have been set
LammpsImpropers: 'atom1' have been set
LammpsImpropers: 'atom2' have been set
LammpsImpropers: 'atom3' have been set
LammpsImpropers: 'atom4' have been set


In [44]:
writer.get_required_datanames()

{'mol'}

In [45]:
writer.get_required_datanames_for_molecule()

set()

In [46]:
writer.set_atom_data(mol=[1]*len(atoms))

LammpsAtoms: 'mol' have been set


In [47]:
bond_types = {
  seq: i+1
  for i, seq in enumerate(writer.get_bond_patterns())
}
bond_types

{(1, 2): 1, (1, 3): 2, (5, 6): 3, (1, 5): 4, (3, 4): 5}

In [48]:
writer.set_bond_types(bond_types)

LammpsBonds: 'type' have been set


In [49]:
angle_types = {
  seq: i+1
  for i, seq in enumerate(writer.get_angle_patterns())
}
angle_types

{(3, 1, 5): 1,
 (6, 5, 6): 2,
 (2, 1, 5): 3,
 (1, 5, 6): 4,
 (1, 3, 4): 5,
 (2, 1, 3): 6}

In [50]:
writer.set_angle_types(angle_types)

LammpsAngles: 'type' have been set


In [51]:
dihedral_types = {
  seq: i+1
  for i, seq in enumerate(writer.get_dihedral_patterns())
}
dihedral_types

{(4, 3, 1, 2): 1, (6, 5, 1, 3): 2, (5, 1, 3, 4): 3, (6, 5, 1, 2): 4}

In [52]:
writer.set_dihedral_types(dihedral_types)

LammpsDihedrals: 'type' have been set


In [53]:
improper_types = {
  seq: i+1
  for i, seq in enumerate(writer.get_improper_patterns())
}
improper_types

{(1, 2, 3, 5): 1, (5, 1, 6, 6): 2, (5, 6, 6, 6): 3}

In [54]:
writer.set_improper_types(improper_types)

LammpsImpropers: 'type' have been set


In [55]:
writer.print_max_bonds_per_atom()
writer.print_max_angles_per_atom()
writer.print_max_dihedrals_per_atom()
writer.print_max_impropers_per_atom()

You might need to set 'extra/bond/per/atom' to: 3
You might need to set 'extra/angle/per/atom' to: 6
You might need to set 'extra/dihedral/per/atom' to: 4
You might need to set 'extra/improper/per/atom' to: 3


In [56]:
writer.write_lammps_data("./lammps_files/data.acetic", mass=True)

'Masses' section was written
'Atoms' section was written
'Bonds' section was written
'Angles' section was written
'Dihedrals' section was written
'Impropers' section was written


* [link to the file](./lammps_files/data.acetic)

In [57]:
writer.write_lammps_molecule("./lammps_files/molecule.acetic", mass=True)

'Coords' section was written
'Types' section was written
'Masses' section was written
'Bonds' section was written
'Angles' section was written
'Dihedrals' section was written
'Impropers' section was written
'Special Bond Counts' and 'Special Bonds' section was written


* [link to the file](./lammps_files/molecule.acetic)

In [58]:
writer.write_lammps_data("./lammps_files/data.acetic2", centering=True, velocity=True, mass=True)

'Masses' section was written
'Atoms' section was written
'Velocities' section was written
'Bonds' section was written
'Angles' section was written
'Dihedrals' section was written
'Impropers' section was written


* [link to the file](./lammps_files/data.acetic2)

## NaCl

In [59]:
atoms = BondedAtoms(bulk("NaCl", "rocksalt", a=5.64, orthorhombic=True))

In [60]:
atoms.get_cell()

array([[3.98808225, 0.        , 0.        ],
       [0.        , 3.98808225, 0.        ],
       [0.        , 0.        , 5.64      ]])

In [61]:
atoms.get_atomic_numbers()

array([11, 17, 11, 17])

In [62]:
atoms.set_types([1, 2, 1, 2])

In [63]:
atoms.get_positions()

array([[0.        , 0.        , 0.        ],
       [1.99404112, 1.99404112, 0.        ],
       [1.99404112, 1.99404112, 2.82      ],
       [0.        , 0.        , 2.82      ]])

In [64]:
atoms.change_max_bonds(6)

In [65]:
length = 2.82

cell = atoms.get_cell()
positions = atoms.get_positions()

for i, j in itertools.combinations(range(len(atoms)), 2):
  r_original = positions[j] - positions[i]
  for ix, iy, iz in itertools.product(*[(-1, 0, 1)]*3):
    r = r_original + ix * cell[0] + iy * cell[1] + iz * cell[2]
    if np.isclose(np.linalg.norm(r), length):
      atoms.add_bond(i, j, img2=(ix, iy, iz))

In [66]:
atoms.get_types()

array([1, 2, 1, 2])

In [67]:
atoms.get_bonds()

array([[[ 1, -1, -1,  0],
        [ 1, -1,  0,  0],
        [ 1,  0, -1,  0],
        [ 1,  0,  0,  0],
        [ 3,  0,  0, -1],
        [ 3,  0,  0,  0]],

       [[-1,  1,  1,  0],
        [-1,  1,  0,  0],
        [-1,  0,  1,  0],
        [-1,  0,  0,  0],
        [ 1,  0,  0, -1],
        [ 1,  0,  0,  0]],

       [[-1,  0,  0,  1],
        [-1,  0,  0,  0],
        [ 1,  0,  0,  0],
        [ 1,  0,  1,  0],
        [ 1,  1,  0,  0],
        [ 1,  1,  1,  0]],

       [[-3,  0,  0,  1],
        [-3,  0,  0,  0],
        [-1,  0,  0,  0],
        [-1,  0, -1,  0],
        [-1, -1,  0,  0],
        [-1, -1, -1,  0]]])

In [68]:
atoms *= 5

In [69]:
writer = LammpsWriter(atoms, atom_style="molecular")

LammpsAtoms: 'id' have been set
LammpsAtoms: 'type' have been set
LammpsAtoms: 'x' have been set
LammpsAtoms: 'y' have been set
LammpsAtoms: 'z' have been set
LammpsAtoms: 'mass' have been set
LammpsBonds: 'id' have been set
LammpsBonds: 'atom1' have been set
LammpsBonds: 'atom2' have been set
LammpsAngles: 'id' have been set
LammpsAngles: 'atom1' have been set
LammpsAngles: 'atom2' have been set
LammpsAngles: 'atom3' have been set
LammpsDihedrals: 'id' have been set
LammpsDihedrals: 'atom1' have been set
LammpsDihedrals: 'atom2' have been set
LammpsDihedrals: 'atom3' have been set
LammpsDihedrals: 'atom4' have been set
LammpsImpropers: 'id' have been set
LammpsImpropers: 'atom1' have been set
LammpsImpropers: 'atom2' have been set
LammpsImpropers: 'atom3' have been set
LammpsImpropers: 'atom4' have been set


In [70]:
writer.get_required_datanames()

{'mol'}

In [71]:
writer.set_atom_data(mol=[0]*len(atoms))

LammpsAtoms: 'mol' have been set


In [72]:
bond_types = {
  seq: i+1
  for i, seq in enumerate(writer.get_bond_patterns())
}
bond_types

{(1, 2): 1}

In [73]:
writer.set_bond_types(bond_types)

LammpsBonds: 'type' have been set


In [74]:
angle_types = {
  seq: i+1
  for i, seq in enumerate(writer.get_angle_patterns())
}
angle_types

{(1, 2, 1): 1, (2, 1, 2): 2}

In [75]:
writer.set_angle_types(angle_types)

LammpsAngles: 'type' have been set


In [76]:
writer.print_max_bonds_per_atom()
writer.print_max_angles_per_atom()
writer.print_max_dihedrals_per_atom()
writer.print_max_impropers_per_atom()

You might need to set 'extra/bond/per/atom' to: 6
You might need to set 'extra/angle/per/atom' to: 15
You might need to set 'extra/dihedral/per/atom' to: 0
You might need to set 'extra/improper/per/atom' to: 0


In [77]:
writer.write_lammps_data("./lammps_files/data.nacl.ba", mass=True)

'Masses' section was written
'Atoms' section was written
'Bonds' section was written
'Angles' section was written


* [link to the file](./lammps_files/data.nacl.ba)

In [78]:
dihedral_types = {
  seq: i+1
  for i, seq in enumerate(writer.get_dihedral_patterns())
}
dihedral_types

{(1, 2, 1, 2): 1}

In [79]:
writer.set_dihedral_types(dihedral_types)

LammpsDihedrals: 'type' have been set


In [80]:
improper_types = {
  seq: i+1
  for i, seq in enumerate(writer.get_improper_patterns())
}
improper_types

{(2, 1, 1, 1): 1, (1, 2, 2, 2): 2}

In [81]:
writer.set_improper_types(improper_types)

LammpsImpropers: 'type' have been set


In [82]:
writer.print_max_bonds_per_atom()
writer.print_max_angles_per_atom()
writer.print_max_dihedrals_per_atom()
writer.print_max_impropers_per_atom()

You might need to set 'extra/bond/per/atom' to: 6
You might need to set 'extra/angle/per/atom' to: 15
You might need to set 'extra/dihedral/per/atom' to: 78
You might need to set 'extra/improper/per/atom' to: 36


In [83]:
writer.write_lammps_data("./lammps_files/data.nacl.badi", mass=True)

'Masses' section was written
'Atoms' section was written
'Bonds' section was written
'Angles' section was written
'Dihedrals' section was written
'Impropers' section was written


* [link to the file](./lammps_files/data.nacl.badi)